In [5]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM #the model class to fit the model

from fetch_lasfm_data import fetch_lastfm

In [6]:
#fetch data and format it
data = fetch_movielens(min_rating=4.0)

#fetch of the data in the right format 
data2 = fetch_lastfm()

Dataset not found, please download the file from:
http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-360K.tar.gz (1.6GB)
then place it in recommender_system_py/data/ and change the file_path
variable on the fetch_lastfm function


In [7]:
#print training and testing data
print(repr(data['train']))
print(repr(data['test']))
#We have two such matrices, a training and a testing set. 
#Both have around 1000 users and 1700 items. We’ll train 
#the model on the train matrix but test it on the test matrix.



<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 49906 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 5469 stored elements in COOrdinate format>


In [8]:
#create modelì
model = LightFM(loss='warp')
# train model
model.fit(data['train'], epochs=30, num_threads=2)

#-----------

model2 = LightFM(loss='warp')
model.fit(data['matrix'], epochs=30, num_threads=2)

KeyError: 'matrix'

In [ ]:
from lightfm.evaluation import precision_at_k

In [ ]:
# Measure of the precision
print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

In [9]:
#Test of the model
def sample_recommendation(model, data, user_ids):

    #number of users and movies in training data
    n_users, n_items = data['train'].shape
    
    #generate recommendations for each user we input
    for user_id in user_ids:
        #movies they already like
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        #movies our model predicts they will be like
        scores = model.predict(user_id, np.arange(n_items))
        #rank them in order of most liked to least
        top_items = data['item_labels'][np.argsort(-scores)]
        
        #print out the results
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)
            
        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)
            
sample_recommendation(model, data, [3, 25, 450])

# ----------------------------------------------

def get_recommendations(model, coo_mtrx, users_ids):
    n_items = coo_mtrx.shape[1]
    
    for user in users_ids:
        scores = model.predict(user, np.arange(n_items))
        top_scores = np.argsort(-scores)[:3]
        
        print('Recomendations for user %s:' % user)

        for x in top_scores.tolist():
            for artist, values in data['artists'].iteritems():
                if int(x) == values['id']:
                    print ('   - %s' % values['name'])

        print ('\n') # Get it pretty


user_1 = raw_input('Select user_1 (0 to %s): ' % data['users'])
user_2 = raw_input('Select user_2 (0 to %s): ' % data['users'])
user_3 = raw_input('Select user_3 (0 to %s): ' % data['users'])
print ('\n') # Get it pretty

get_recommendations(model, data['matrix'], [user_1, user_2, user_3])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Contact (1997)
        Starship Troopers (1997)
     Recommended:
        Scream (1996)
        Dante's Peak (1997)
        Scream 2 (1997)
User 25
     Known positives:
        Dead Man Walking (1995)
        Star Wars (1977)
        Fargo (1996)
     Recommended:
        L.A. Confidential (1997)
        Fargo (1996)
        Contact (1997)
User 450
     Known positives:
        Contact (1997)
        George of the Jungle (1997)
        Event Horizon (1997)
     Recommended:
        G.I. Jane (1997)
        Dante's Peak (1997)
        Peacemaker, The (1997)


NameError: name 'raw_input' is not defined